In [1]:
import json
import gzip
import numpy as np
import pandas as pd

### Load all pages

In [2]:
with gzip.open('all_pages.json.gz', 'rt', encoding='UTF-8') as zipfile:
    data = json.load(zipfile)

### Extract all titles without words in brackets

In [3]:
def newtitles(data):
    new_titles = []

    for i in range(0, len(data)):
        
        title = data[i]['title']
        new_title = ''
        word = ''
        j = 0
        braket = False 
        
        while j < len(title):
            char = title[j]
            
            if char == ')': 
                braket = False
                word = ''
            
            if char == '(':
                braket = True
                if len(word) > 2:
                    new_title = new_title + word.lower() + ' '
                    word = ''
                    
                
            elif braket: braket = True
                
            elif char.isalpha(): word = word + char
                
            elif char == ' ' or char == '-' or char == '\'':
                if len(word) > 1: 
                    new_title = new_title + word.lower() + ' '
                word = ''
                
            j = j + 1
            
        if len(word) > 2:
            new_title = new_title + word.lower() + ' '
            
        new_titles.append(new_title)
    
    return new_titles

In [4]:
newT = newtitles(data)

### Creation of names (first + last word)

In [5]:
def names(data):
    names = []
    for i in range(len(data)):
        name = []
        word = ''
        for j in range(len(data[i])):
            char = data[i][j]
            if char == ' ':
                name.append(word)
                word = ''
            else: word = word + char
        if len(name) > 1:
            names.append(name[0] + ' '+ name[-1])
        else: names.append('')

    return names    

In [6]:
names = names(newT)

### Creation of data Frame with new titles

In [7]:
dF = pd.DataFrame(data)
dF['new_title'] = newT
dF['name'] = names

### Find all new_titles and names that are dublicated

In [14]:
def dublications(dF, col):
    #delete rows with new_title = ''
    cleaned_dF = dF[dF[col] != '']
    #search dublications in new_title and keep those rows
    dubli_dF = cleaned_dF[cleaned_dF.duplicated(subset = col, keep = False)]
    #return the dubli_dF which is sorted by new_title
    return dubli_dF.sort_values(by = col)

In [15]:
title_dub_dF = dublications(dF, 'new_title')
name_dub_dF = dublications(dF, 'name')

### Make list of homonyms

In [16]:
def homonyms(dF, col):
    h_list = []
    homonyms = []
    homonym = dF.iloc[0][col]
    for i in range(len(dF)):
        if dF.iloc[i][col] == homonym:
            homonyms.append(dF.iloc[i]['title'])
        else:
            h_list.append(homonyms)
            homonyms = [dF.iloc[i]['title']]
            homonym = dF.iloc[i][col]
    h_list.append(homonyms)
    
    return h_list 

In [17]:
title_homonomys = homonyms(title_dub_dF, 'new_title')
name_homonomys = homonyms(name_dub_dF, 'name')

In [18]:
filearray_titles = np.array(title_homonomys, dtype = "object")
np.save('title_homonomys.npy', filearray_titles)

filearray_names = np.array(name_homonomys, dtype = "object")
np.save('name_homonomys.npy', filearray_names)

### Load homonyms lists (+ example)

In [20]:
title_h = np.load('title_homonomys.npy', allow_pickle=True)
name_h = np.load('name_homonomys.npy', allow_pickle=True)

In [21]:
for i in range(500,505):
    print(title_h[i])

['Adama Traoré', 'Adama Traoré (aeb16e2)']
['Adamiste (bois de sciage)', 'Adamiste (propriétaire, Strasbourg)', 'Adamiste (propriétaire, Saint-Denis)', 'Adamiste (propriétaire)', 'Adamiste (menuisier en bâtiments)', 'Adamiste (bois de sciage, boul. de Sébastopol)', 'Adamiste (bois de sciage, boul. de Strasbourg)', 'Adamiste (Vve)', 'Adamiste (Vve) (propriétaire, 53 boulevard Strasbourg)', 'Adamiste (Vve) (propriét, 53 boulevard de Strasbourg)', "Adamiste (Page d'homonymie)", 'Adamiste']
['Adamiste (Vve) - propriétaire', 'Adamiste (Vre) - propriétaire', 'Adamiste (vve) - propriétaire', 'Adamiste (yve) - propriétaire', 'Adamiste (Yve) - propriétaire']
['Adams (F.)', 'Adams (Joseph)', 'T. S. Adams']
['Adams (Mme) - café-brasserie', 'Adams (Mime) - café-brasserie']


In [23]:
for i in range(1000,1005):
    print(name_h[i])

['Alain Michel', 'Alain Michel (e9ebcd9)', 'Alain-René Michel']
['Alain Milon', 'Alain Milon (69e719e)']
['Alain Monnier', 'Alain Monnier (0d12356)']
['Alain Moreau (6008888)', 'Alain Moreau (a45968b)', 'Alain Moreau (5f78cb2)', 'Alain Moreau', 'Alain Moreau (47a6236)']
['Alain Morel', 'Alain Morel (5b11e32)', 'Alain Morel (88a48cf)', 'Alain Morel (f11f49d)']
